In [ ]:
#import library

import cv2 as cv #for image processing
import time #for current time control
import numpy as np
print(cv.__version__)

In [ ]:
#obtain camera meta-information

capture = cv.VideoCapture('2021.11.13.15.28.5.avi') #open camera
ret, frame = capture.read() #read camera's input
width = capture.get(cv.CAP_PROP_FRAME_WIDTH) #obtain width
height = capture.get(cv.CAP_PROP_FRAME_HEIGHT) #obtain height
fps = capture.get(cv.CAP_PROP_FPS) #obtain fps
#fps=8
print('width {0}, height {1}, fps {2}'.format(width, height, fps)) #print information

In [ ]:
#configuration of RGB recoding

fourcc = cv.VideoWriter_fourcc(*'DIVX') #for codec
#out = cv.VideoWriter(filename, fourcc, fps, (int(width), int(height))) #class for recoding

In [ ]:
#main body. it describes all task for each frame.

start_time=time.time() #obtain current time for basis

flag = 1 #flag for record
delay = int(1000/fps) #delay based on fps
isFirst = True

trackers = [
            cv.TrackerCSRT_create,
             cv.TrackerGOTURN_create,
            cv.TrackerKCF_create,
    cv.TrackerMIL_create,
           ]
trackerIdx = 0 
tracker = None
count=0
win_name = 'Tracking APIs'
#print(int(time.time()-start_time))
while True: #repeat infinitely
    ret, frame_original = capture.read() #read a frame
    #frame = cv.cvtColor(frame_original, cv.COLOR_BGR2HSV)
    frame = cv.inRange(frame_original, (0,0,0), (50,50,50))
    frame_baby = cv.inRange(frame_original, (45,45,90), (70,70,120))
    #cv.imshow("temp", frame_baby)
    #kernel = np.ones((5, 5), np.unit8)
    frame_baby = cv.morphologyEx(frame_baby, cv.MORPH_OPEN, kernel=(10,10), iterations = 2)
    frame = cv.morphologyEx(frame, cv.MORPH_OPEN, kernel=(10,10), iterations = 2)
    frame = cv.dilate(frame, (5,5), iterations = 5)
    #컨투어
    contour, hierarchy = cv.findContours(frame, cv.RETR_EXTERNAL, \
                                                 cv.CHAIN_APPROX_NONE)
    # 가장 바깥쪽 컨투어에 대해 꼭지점 좌표만 반환 ---④
    contour2, hierarchy = cv.findContours(frame_baby, cv.RETR_EXTERNAL, \
                                                    cv.CHAIN_APPROX_SIMPLE)
    # 각각의 컨투의 갯수 출력 ---⑤
    #print('도형의 갯수: %d(%d)'% (len(contour), len(contour2)))
    for cnt in contour:
        cv.drawContours(frame_original, contour, -1, (0,255,0), 4)
    for cnt in contour2:
        cv.drawContours(frame_original, contour2, -1, (0,255,0), 4)
    #cv.imshow(image_draw)# 꼭지점 좌표만을 갖는 컨투어 그리기, 초록색  ---⑦
    # 모든 좌표를 갖는 컨투어 그리기, 초록색  ---⑥
    
    #cv2.drawContours(img2, contour2, -1, (0,255,0), 4)
    #컨투어
    #cv.imshow("temp", frame_baby)
    #count+=1
    #print(count, int(time.time()-start_time))
    current_time=[time.localtime().tm_year,time.localtime().tm_mon, \
                  time.localtime().tm_mday, time.localtime().tm_hour, time.localtime().tm_min, \
                  time.localtime().tm_sec]   #obtain current time
    current_time_second=time.time()
    
    img_draw = frame_original.copy()
    #cv.imshow("rer", img_draw)
    #cv.drawContours(img_draw, contour, -1, (0,255,0), 4)
    #cv.imshow(img_draw)# 꼭지점 좌표만을 갖는 컨투어 그리기, 초록색  ---⑦
    #cv.putText(img_draw, str(current_time[0])+"."+str(current_time[1])+"."+str(current_time[2])+"."+str(current_time[3])+"."+\
    #                                   str(current_time[4])+"."+str(current_time[5])+"   5 Mice", \
    #        (150,20), cv.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,0),2,cv.LINE_AA)
    if tracker is None:
        pass
    else:
        ok, bbox = tracker.update(frame)   # update new frame
        if ok == True:
            (x,y,w,h) = bbox
        cv.rectangle(img_draw, (int(x), int(y)), (int(x + w), int(y + h)), \
                          (255,0,0), 2, 1)
        ok1, bbox1 = tracker1.update(frame_baby)   # update new frame
        if ok == True:
            (x1,y1,w1,h1) = bbox1
        else:
            isInit = tracker1.init(frame, (x1,y1,w1,h1))
        cv.rectangle(img_draw, (int(x1), int(y1)), (int(x1 + w1), int(y1 + h1)), \
                          (255,128,0), 2, 1)
        #print(ok1, bbox1)
        #if (ok1 == False):
            
        #ok2, bbox2 = tracker2.update(frame)   # update new frame
        #(x2,y2,w2,h2) = bbox2
        #cv.rectangle(img_draw, (int(x2), int(y2)), (int(x2 + w2), int(y2 + h2)), \
        #                  (255,255,0), 2, 1)
        #ok3, bbox3 = tracker3.update(frame)   # update new frame
        #(x3,y3,w3,h3) = bbox3
        #cv.rectangle(img_draw, (int(x3), int(y3)), (int(x3 + w3), int(y3 + h3)), \
        #                  (0,255,0), 2, 1)
        #ok4, bbox4 = tracker4.update(frame)   # update new frame
        #(x4,y4,w4,h4) = bbox4
        #cv.rectangle(img_draw, (int(x4), int(y4)), (int(x4 + w4), int(y4 + h4)), \
        #                  (0,0,255), 2, 1)
    #if count % 3 ==0:    
    cv.imshow(win_name, img_draw)
    cv.imshow("er",frame_baby)
    
    if isFirst: #first time
        isFirst = False
        roi = cv.selectROIs(win_name, frame_original, False)  
                
        tracker = trackers[trackerIdx]()    
        isInit = tracker.init(frame, roi[0])
        tracker1 = trackers[trackerIdx]()    
        isInit1 = tracker1.init(frame, roi[1])
        #tracker2 = trackers[trackerIdx]()    
        #isInit = tracker2.init(frame, roi[2])
        #tracker3 = trackers[trackerIdx]()    
        #isInit = tracker3.init(frame, roi[3])
        #tracker4 = trackers[trackerIdx]()    
        #isInit = tracker4.init(frame, roi[4])
    
    ####### recoding part
    
    if int(current_time_second-start_time) % 3600 == 0 and flag == 1: #per 1 hour re-name video file.
        filename =str(current_time[0])+"."+str(current_time[1])+"."+str(current_time[2])+"."+str(current_time[3])+"."+\
                                       str(current_time[4])+"."+str(current_time[5])
        out = cv.VideoWriter(filename+'.avi', fourcc, fps, (int(width), int(height))) #class for recoding
        
        flag=0
        
    elif int(current_time_second-start_time) % 3599 == 0: #check 1 hour
        flag=1
    
    out.write(img_draw) # write a frame in a video 
    ###### /recoding part
    #out.write(frame)
    
    
    """
    imgray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    ret, thr = cv.threshold(imgray, 70, 255, cv.THRESH_BINARY)


    countors, _ = cv.findContours(thr, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(frame, countors, -1, (0, 255, 0), 1)

    cv.imshow('thr', thr)
    cv.imshow('contours', frame)
    
    #cv.imshow("VideoFrame", frame)
    """
    key = cv.waitKey(1)  # if ESC key is pressed, then exit 
    if key ==27:
        break

    
capture.release()  #terminate all the code.
cv.destroyAllWindows()



In [ ]:
import serial

ser = serial.Serial("COM30", 9600, timeout=1)
ser1 = serial.Serial("COM31", 9600, timeout=1)
ser2 = serial.Serial("COM32", 9600, timeout=1)
ser3 = serial.Serial("COM33", 9600, timeout=1)
ser4 = serial.Serial("COM34", 9600, timeout=1)
ser5 = serial.Serial("COM35", 9600, timeout=1)
ser6 = serial.Serial("COM36", 9600, timeout=1)
ser7 = serial.Serial("COM37", 9600, timeout=1)
ser8 = serial.Serial("COM38", 9600, timeout=1)
ser9 = serial.Serial("COM39", 9600, timeout=0)
ser10 = serial.Serial("COM40", 9600, timeout=0)
ser11 = serial.Serial("COM41", 9600, timeout=0)
ser12 = serial.Serial("COM42", 9600, timeout=0)
ser13 = serial.Serial("COM43", 9600, timeout=0)
ser14 = serial.Serial("COM44", 9600, timeout=0)
ser15 = serial.Serial("COM45", 9600, timeout=0)
#ser.timeout = 1
while True:
    #print("insert op :", end=' ')
    print("R0: ", ser.readline())
    print("R1: ", ser1.readline())
    print("R2: ", ser2.readline())
    print("R3: ", ser3.readline())
    print("R4: ", ser4.readline())
    print("R5: ", ser5.readline())
    print("R6: ", ser6.readline())
    print("R7: ", ser7.readline())
    print("R8: ", ser8.readline())
    print("R9: ", ser9.readline())
    print("R10: ", ser10.readline())
    print("R11: ", ser11.readline())
    print("R12: ", ser12.readline())
    print("R13: ", ser13.readline())
    print("R14: ", ser14.readline())
    print("R15: ", ser15.readline())
    print("----------------")
    #if op == 'q':
    #    ser.close()